# Setup

In [ ]:
!pip install graphdatascience

In [ ]:
# Import the client
from graphdatascience import GraphDataScience

# Replace with the actual URI, username, and password
AURA_CONNECTION_URI = "neo4j+s://xxxxxxxx.databases.neo4j.io"
AURA_USERNAME = "neo4j"
AURA_PASSWORD = ""

# Configure the client with AuraDS-recommended settings
gds = GraphDataScience(
    AURA_CONNECTION_URI,
    auth=(AURA_USERNAME, AURA_PASSWORD),
    aura_ds=True
)

In [ ]:
gds.run_cypher("""
    CREATE
      (home:Page {name:'Home'}),
      (about:Page {name:'About'}),
      (product:Page {name:'Product'}),
      (links:Page {name:'Links'}),
      (a:Page {name:'Site A'}),
      (b:Page {name:'Site B'}),
      (c:Page {name:'Site C'}),
      (d:Page {name:'Site D'}),

      (home)-[:LINKS {weight: 0.2}]->(about),
      (home)-[:LINKS {weight: 0.2}]->(links),
      (home)-[:LINKS {weight: 0.6}]->(product),
      (about)-[:LINKS {weight: 1.0}]->(home),
      (product)-[:LINKS {weight: 1.0}]->(home),
      (a)-[:LINKS {weight: 1.0}]->(home),
      (b)-[:LINKS {weight: 1.0}]->(home),
      (c)-[:LINKS {weight: 1.0}]->(home),
      (d)-[:LINKS {weight: 1.0}]->(home),
      (links)-[:LINKS {weight: 0.8}]->(home),
      (links)-[:LINKS {weight: 0.05}]->(a),
      (links)-[:LINKS {weight: 0.05}]->(b),
      (links)-[:LINKS {weight: 0.05}]->(c),
      (links)-[:LINKS {weight: 0.05}]->(d)
""")

""


In [ ]:
g, result = gds.graph.project(
    "example-graph",
    "Page",
    "LINKS",
    relationshipProperties="weight"
)

print(result)

nodeProjection                {'Page': {'label': 'Page', 'properties': {}}}
relationshipProjection    {'LINKS': {'orientation': 'NATURAL', 'aggregat...
graphName                                                     example-graph
nodeCount                                                                 8
relationshipCount                                                        14
projectMillis                                                             5
Name: 0, dtype: object


# Execution modes

## Stats mode

In [ ]:
result = gds.pageRank.stats(
    g,
    maxIterations=20,
    dampingFactor=0.85
)

print(result)

ranIterations                                                            20
didConverge                                                           False
centralityDistribution    {'p99': 3.2156810760498047, 'min': 0.327856063...
postProcessingMillis                                                     12
preProcessingMillis                                                       0
computeMillis                                                            28
configuration             {'maxIterations': 20, 'sourceNodes': [], 'scal...
Name: 0, dtype: object


## Stream mode

In [ ]:
results = gds.pageRank.stream(
    g,
    maxIterations=20,
    dampingFactor=0.85
)

print(results)

   nodeId     score
0       0  3.215682
1       1  1.054270
2       2  1.054270
3       3  1.054270
4       4  0.327858
5       5  0.327858
6       6  0.327858
7       7  0.327858


## Mutate mode

In [ ]:
result = gds.pageRank.mutate(
    g,
    mutateProperty="pageRankScore",
    maxIterations=20,
    dampingFactor=0.85
)

print(result)

mutateMillis                                                              1
nodePropertiesWritten                                                     8
ranIterations                                                            20
didConverge                                                           False
centralityDistribution    {'p99': 3.2156810760498047, 'min': 0.327856063...
postProcessingMillis                                                     16
preProcessingMillis                                                       0
computeMillis                                                            26
configuration             {'maxIterations': 20, 'sourceNodes': [], 'scal...
Name: 0, dtype: object


## Write mode

In [ ]:
result = gds.pageRank.write(
    g,
    writeProperty="pageRankScore",
    maxIterations=20,
    dampingFactor=0.85
)

print(result)

writeMillis                                                               9
nodePropertiesWritten                                                     8
ranIterations                                                            20
didConverge                                                           False
centralityDistribution    {'p99': 3.2156810760498047, 'min': 0.327856063...
postProcessingMillis                                                     12
preProcessingMillis                                                       0
computeMillis                                                            26
configuration             {'maxIterations': 20, 'writeConcurrency': 4, '...
Name: 0, dtype: object


# Cleanup

In [ ]:
result = gds.graph.drop(g)
print(result)

gds.run_cypher("""
    MATCH (n)
    DETACH DELETE n
""")

graphName                                                example-graph
database                                                         neo4j
memoryUsage                                                           
sizeInBytes                                                         -1
nodeCount                                                            8
relationshipCount                                                   14
configuration                                                       {}
density                                                           0.25
creationTime                       2022-03-29T13:59:06.598241000+00:00
modificationTime                   2022-03-29T14:01:18.194290000+00:00
schema               {'relationships': {'LINKS': {'weight': 'Float ...
Name: 0, dtype: object


""
